In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hppfcl
import numpy as np
import time

import jax
from jax import jacfwd, jacrev, vmap, jit
from jax import numpy as jnp
jax.config.update('jax_platform_name', 'cpu')


In [24]:
A = np.array([[1,0],[0,1.0]])

%timeit jnp.linalg.eigvals(A).block_until_ready()
if(jnp.all(jax.numpy.linalg.eigh(A)[0] > 0)):
    print("Positive definite")

4.8 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Positive definite


In [32]:
def test_A(x):
    def in_b(x):
        return x+1
    
    def in_c(x):
        y = in_b(x)+1
        print()
        def in_in():
            return in_b(y)
        
        return in_in()
    
    return in_c(x)

test_A(1)

4

In [3]:
verts = hppfcl.StdVec_Vec3f()
faces = hppfcl.StdVec_Triangle()
verts.extend(
    [
        np.array([0.5, 0.5, 0]),
        np.array([0.5, -0.5, 0]),
        np.array([-0.5, 0.5, 0]),
        np.array([-0.5, -0.5, 0]),
        np.array([0.5, 0.5, 0.5]),
        np.array([0.5, -0.5, 0.5]),
        np.array([-0.5, 0.5, 0.5]),
        np.array([-0.5, -0.5, 0.5]) ,
        # np.array([0, 0, -0.5]),
        # np.array([0, 0, 0.5]),
    ]
)
c = hppfcl.Convex.convexHull(verts, True, None)
c.points()
b = hppfcl.Box(1, 1, 1)

obj1 = hppfcl.CollisionObject(c, np.eye(3), np.array([0,0,0]))
obj2 = hppfcl.CollisionObject(b, np.eye(3), np.array([-2.0,2.0,0]))

request = hppfcl.DistanceRequest()
result = hppfcl.DistanceResult()
hppfcl.distance(obj2, obj1, request, result)
print(result.min_distance)




1.4142135623730951


In [93]:
def dis_func(x, y):
    # collision object
    obj1 = hppfcl.CollisionObject(hppfcl.Box(2, 2, 2), np.eye(3), np.array([0,0,0]))
    obj2 = hppfcl.CollisionObject(hppfcl.Box(1, 1, 1), np.eye(3), np.array([x,y,1.5]))

    request = hppfcl.DistanceRequest(True, 0.1, 0.1)
    result = hppfcl.DistanceResult()
    hppfcl.distance(obj1, obj2, request, result)
    print(result.getNearestPoint1())
    print(result.getNearestPoint2())
    return result.min_distance

def collid_func(x, y):
    # collision object
    obj1 = hppfcl.CollisionObject(hppfcl.Box(1, 1, 1), np.eye(3), np.array([0,0,0]))
    obj2 = hppfcl.CollisionObject(hppfcl.Box(1, 1, 1), np.eye(3), np.array([x,y,0]))

    request = hppfcl.CollisionRequest()
    result = hppfcl.CollisionResult()
    hppfcl.collide(obj1, obj2, request, result)
    return result.distance_lower_bound

print(dis_func(-1.5,-1.5))


[-1. -1.  1.]
[-1. -1.  1.]
0.0


In [ ]:
def test(a, b):
    return a**3+b**3+3*a*b

In [ ]:
import numpy as np

N = 1000
D = 2   
e = 1e-5
z = np.random.randn(N, D)


np.mean(test(z[:,0]*e+1, z[:,1]*e+1)[...,np.newaxis]*z, axis=0)





array([0.016788  , 0.05044002])

In [ ]:
jax.grad(test, argnums=(0,1))(1.0,1.0)

(Array(6., dtype=float32, weak_type=True),
 Array(6., dtype=float32, weak_type=True))